Link Extraction
===

Extracts links from journals, comments, and guestbook entries.

Currently written up with pseudo-code to operate on the journals.

This notebook is written with a workflow that made sense to Zach, but it can definitely be changed and updated: new functions can be written, old functions can be deleted or merged, etc.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import re
import pandas as pd
import numpy as np

from collections import Counter, defaultdict
import sqlite3
from html.parser import HTMLParser
from tqdm import tqdm, tqdm_notebook
from datetime import datetime

import matplotlib.pyplot as plt

#### Load the site data

This provides the list of site URL 'names'.

In [3]:
site_metadata_working_dir = "/home/srivbane/shared/caringbridge/data/derived/site_metadata"
site_metadata_filepath = os.path.join(site_metadata_working_dir, "site_metadata_with_text.feather")
site_df = pd.read_feather(site_metadata_filepath)
len(site_df)

840943

In [ ]:
site_df.head()

In [5]:
# note there are a very small number of duplicate sites
duplicate_sites = site_df[site_df.duplicated(subset='name', keep=False)][['site_id', 'name', 'title', 'created_at', 'visits']]
len(duplicate_sites)

14

In [6]:
# read valid sites being included in this study
valid_site_ids = set()
data_selection_dir = "/home/srivbane/shared/caringbridge/data/projects/sna-social-support/data_selection"
with open(os.path.join(data_selection_dir, "valid_site_ids.txt"), 'r') as infile:
    for line in infile:
        site_id = line.strip()
        if site_id == "":
            continue
        else:
            valid_site_ids.add(int(site_id))
len(valid_site_ids)

340414

In [7]:
# note that 5 of the 'duplicate name' site ids exist in the sample for this study
len(set(site_df.site_id[site_df.duplicated(subset='name', keep=False)]) & set(valid_site_ids))

5

In [8]:
# take a look at the duplicate sites, noting different creation dates and different titles
# in one case, we include one site but not the other in the SNA sample
# these sites need to be investigated more specifically
duplicate_sites['Human-readable creation date'] = duplicate_sites.created_at.apply(lambda created_at: str(datetime.utcfromtimestamp(created_at / 1000)))
duplicate_sites['In SNA sample?'] = duplicate_sites.site_id.apply(lambda site_id: site_id in valid_site_ids)
duplicate_sites.sort_values(by='name')

,site_id,name,title,created_at,visits,Human-readable creation date,In SNA sample?
833940,1327860,abramsmcbride68,abramsmcbride68,1548780285000,2,2019-01-29 16:44:45,False
833941,1327861,abramsmcbride68,abramsmcbride68,1548780285000,6,2019-01-29 16:44:45,False
826885,1320805,holgersencantrell9,holgersencantrell9,1547397974000,1,2019-01-13 16:46:14,False
826886,1320806,holgersencantrell9,holgersencantrell9,1547397974000,15,2019-01-13 16:46:14,False
837110,1331030,jarvisjarvis51,jarvisjarvis51,1549352727000,0,2019-02-05 07:45:27,False
837111,1331031,jarvisjarvis51,jarvisjarvis51,1549352727000,5,2019-02-05 07:45:27,False
373939,378110,jdavis,Jacob Davis,1313268066000,2189,2011-08-13 20:41:06,True
434931,506751,jdavis,Jacob Davis,1332557581000,812,2012-03-24 02:53:01,True
453865,546843,judyfields,Judy Fields,1338930911000,97,2012-06-05 21:15:11,True
544177,728921,judyfields,Judy Goldsmith Fields,1369855841000,8,2013-05-29 19:30:41,False


In [9]:
# random selection of CaringBridge site URLs
# these are what we need to match textual links to in order to build a network!
np.random.choice(site_df.name, 10, replace=False).tolist()

['susanssite',
 'heartplate66rosemary',
 'allisonguthrie',
 'ninomartino',
 'chfintheicu',
 'tiakrier',
 'mindylovesjack',
 'teamridgeway',
 'haleymaydak',
 'tinayee']

In [10]:
# 'site_names' is a set containing all of the valid CaringBridge site names
# notably including both sites in our sample and all other sites as well
site_names = set(site_df.name)
len(site_names)

840936

#### Iterature through the journal data looking for links

In [11]:
# this returns the sqlite database connection that contains the journal update texts
def get_journal_text_db():
    journal_text_filepath = "/home/srivbane/shared/caringbridge/data/projects/caringbridge_core/journal.sqlite"
    db = sqlite3.connect(
            journal_text_filepath,
            detect_types=sqlite3.PARSE_DECLTYPES
    )
    db.row_factory = sqlite3.Row
    return db

In [21]:
# this function connects to the database, makes a query, and passes the cursor to iterate_cursor
def connect_and_iterate():
    try:
        db = get_journal_text_db()
        cursor = db.cursor()

        cursor.execute("SELECT * FROM journalText")  # TODO Remove this limit to get all of the sites!
        iterate_cursor(cursor)

    finally:  # always do this with these databases!!
        db.close()

# given a database cursor, this function extracts the text and passes it to get_link_texts
def iterate_cursor(cursor, total=19137078):
    for row in tqdm(cursor, total=total):
        body_text = str(row['body'])
        site_id = row['site_id']
        journal_oid = row['journal_oid']
        link_texts = get_link_texts(body_text)
        for txt in link_texts:
            # TODO From each text link, should extract the URL slug and verify if it is in the set of site_names
            if 'caringbridge' not in txt: #every valid link we care about must be a caringbridge link
                spam.append(txt)
                continue
            words = txt.split('/')
            #Check if one slug of the URL contains a valid site name 
            for item in words:
                if 'caringbridge' in item or 'visit' in item:
                    continue
                if  'al' in item or 'www2' in item or 'europe' in item:
                    spam.append(txt)
                    break
                if item in site_names:
                        name = item
                        link = {
                            'site_id': site_id,
                            'journal_oid': journal_oid,
                            'link': txt,
                            'site_name': name
                        }
                        links.append(link)
                                               
            

In [22]:
# this returns a list of string objects that correspond to the links in the text
# I'll note that what we really want is the site 'name' in the URL, but this implementation makes no attempt to extract that name
def get_link_texts(text):
    # TODO Implement me to find all links in the text, whether they are explicit links or just link mentions
    # e.g. should match both an HTML hyperlink or a plain-text copy-pasted link.
    # The existing implementation is a very simple one, and likely misses lots of links.
    extracted_links = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',text)
    links = []
    for item in extracted_links:
        #strip href of formatting
        if '>' in item:
            item = item[:item.find('>')-1]
        links.append(item)
    return links

In [27]:
links = []
spam = []
connect_and_iterate()
print("Valid links: ",len(links))
print("Invalid links: ",len(spam))
print("Percentage of valid links: ",(len(links)/ (len(links) + len(spam))))

100%|██████████| 19137078/19137078 [03:54<00:00, 81440.92it/s] 

Valid links:  101923
Invalid links:  3735813
Percentage of valid links:  0.02655810613340782


In [30]:
links[:10]

[{'site_id': 59067,
  'journal_oid': '51be059d6ca004aa0700cc60',
  'link': 'http://www.caringbridge.org/visit/zack',
  'site_name': 'zack'},
 {'site_id': 59067,
  'journal_oid': '51be059d6ca004aa0700cca8',
  'link': 'http://www.caringbridge.org/visit/kennymoffit',
  'site_name': 'kennymoffit'},
 {'site_id': 59067,
  'journal_oid': '51be059d6ca004aa0700cdb2',
  'link': 'http://www.caringbridge.org/visit/nariahlynn',
  'site_name': 'nariahlynn'},
 {'site_id': 176,
  'journal_oid': '51bdf3e26ca004924e00835f',
  'link': 'http://www.caringbridge.org/visit/cristinas',
  'site_name': 'cristinas'},
 {'site_id': 59067,
  'journal_oid': '51be059d6ca004aa0700d8b8',
  'link': 'http://www.caringbridge.org/visit/loganbaugher',
  'site_name': 'loganbaugher'},
 {'site_id': 59067,
  'journal_oid': '51be059d6ca004aa0700d8bc',
  'link': 'http://www.caringbridge.org/visit/davidcharlton',
  'site_name': 'davidcharlton'},
 {'site_id': 59067,
  'journal_oid': '51be059d6ca004aa0700d8bc',
  'link': 'http://www

In [25]:
links_df = pd.DataFrame(links)
links_df.head()

,site_id,journal_oid,link,site_name
0,59067,51be059d6ca004aa0700cc60,http://www.caringbridge.org/visit/zack,zack
1,59067,51be059d6ca004aa0700cca8,http://www.caringbridge.org/visit/kennymoffit,kennymoffit
2,59067,51be059d6ca004aa0700cdb2,http://www.caringbridge.org/visit/nariahlynn,nariahlynn
3,176,51bdf3e26ca004924e00835f,http://www.caringbridge.org/visit/cristinas,cristinas
4,59067,51be059d6ca004aa0700d8b8,http://www.caringbridge.org/visit/loganbaugher,loganbaugher


In [26]:
# Write the resulting links dataframe to a CSV file
working_dir = "/home/srivbane/shared/caringbridge/data/projects/sna-social-support/data_pulling"
links_df.to_csv(os.path.join(working_dir, 'journal_intersite_links.csv'))
print("Finished.")

Finished.
